In [7]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json

from functions import (get_sector, changes_from_press, 
                       get_df, normalise_to_index, save_data)
columns = ['date', 'press title', 'url', '1d change', 'index_price','norm_price']
past_last_date = False

In [9]:
data = []
offset = 0
not_count = 0

ticker = 'rtx'

sector = get_sector(ticker)

index_ticker = '^GSPC'
# select by value 

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)

year = datetime.today().year

# create break
while True:
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"}
    
    s = requests.Session()
    html = requests.get(f'https://raytheon.mediaroom.com/index.php?o={offset}&s=43&year={year}', headers=headers)
    
    soup = BeautifulSoup(html.content) 

    articles = soup.find_all('div',attrs={'class':'item'})
    
    if not articles:
        if year < 2000:
            break
        offset = 0
        year = year - 1
    
    for article in articles:
        date = article.find('div', attrs={'item_date'}).text.lstrip().rstrip()
        title = article.find('a').text.lstrip().rstrip() 
        url = article.find('a').get('href')
        
        date = datetime.strptime(date, '%b %d, %Y').date()
        if date < datetime(2014, 1, 1).date():
            past_last_date = True
            break
        
        pct_change = changes_from_press(stock_hist, date, 1)
        
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([date, title, url, pct_change, index_price, norm_price])
    if past_last_date == True: break
    offset = offset + 25
    
save_data(data, columns, ticker, sector)

In [2]:
import yahoo_fin.stock_info as si


aapl_earnings_hist = si.get_earnings_history("aapl")


TypeError: string indices must be integers

In [1]:
aapl_earnings_hist

NameError: name 'aapl_earnings_hist' is not defined